In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ms
[nltk_data]     Links\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [4]:
#data1=pd.read_csv(r"C:\Users\Mohammad Noor Malik\Downloads\IMDB data.csv")
data=pd.read_excel('Movies with reviews.xlsx') 
data.head()



,Name of Movie,Year,Genre,Rating,review,sentiment
0,Avatar,2009.0,Action|Adventure|Fantasy|Sci-Fi,7.9,Avatar is one of the best films of the year. T...,positive
1,Pirates of the Caribbean: At World's End,2007.0,Action|Adventure|Fantasy,7.1,"this movie is amazing. It is original, full of...",positive
2,Spectre,2015.0,Action|Adventure|Thriller,6.8,I wasted 3 important hours of my life watching...,negative
3,The Dark Knight Rises,2012.0,Action|Thriller,8.5,This movie is a work of art. The finest sequel...,positive
4,Star Wars: Episode VII - The Force Awakens ...,2015.0,Documentary,7.1,This is by the far worst piece of cr4p I've ev...,negative


In [8]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')


cleaning reviews

In [12]:

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', str(text))
    #x = re.sub('[^a-zA-Z]', '', str(x))

#cleaning the text
def denoise_text(text):
    text = remove_between_square_brackets(text)
    return text

data['review']=data['review'].apply(remove_between_square_brackets)

In [14]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)
data.head(7)

,Name of Movie,Year,Genre,Rating,review,sentiment
0,Avatar,2009.0,Action|Adventure|Fantasy|Sci-Fi,7.9,Avatar is one of the best films of the year Th...,positive
1,Pirates of the Caribbean: At World's End,2007.0,Action|Adventure|Fantasy,7.1,this movie is amazing It is original full of t...,positive
2,Spectre,2015.0,Action|Adventure|Thriller,6.8,I wasted 3 important hours of my life watching...,negative
3,The Dark Knight Rises,2012.0,Action|Thriller,8.5,This movie is a work of art The finest sequel ...,positive
4,Star Wars: Episode VII - The Force Awakens ...,2015.0,Documentary,7.1,This is by the far worst piece of cr4p Ive eve...,negative
5,John Carter,2012.0,Action|Adventure|Sci-Fi,6.6,sadlythis is not the best of the movies to wat...,negative
6,Spider-Man 3,2007.0,Action|Adventure|Romance,6.2,Tobie was is and will be the best spidie in th...,positive


In [15]:
data.to_excel('final project.xlsx')

In [16]:
stop=set(stopwords.words('english'))
print(stop)

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
data['review']=data['review'].apply(remove_stopwords)

{"shan't", "you'd", 'being', 'an', 've', 'hadn', 'who', 'were', "wasn't", 'above', 'ourselves', 'they', 'hers', 'mightn', 'theirs', 'our', 'have', 'couldn', 'not', 'now', 'its', 'no', 'just', 'his', 'again', 'from', 'that', 'down', "weren't", 'will', 'are', 'more', 'only', 'during', 'did', "needn't", 'until', 'both', 'a', 'and', 'so', "couldn't", 'me', 'about', 'do', 'once', "you're", "you've", 'ain', 'i', 'to', 'most', 'has', 'mustn', 'shan', 'but', 'if', 'how', 'you', 'myself', 'the', 'into', 'in', 'yourselves', 'before', 'own', 'those', 'haven', 'any', 'o', 'with', 'my', 'nor', 'between', 'wouldn', 'each', "don't", 'm', 'himself', 'their', "shouldn't", 'some', 'or', 'yourself', 'your', "mightn't", 'aren', 'on', 'll', 'don', 'through', 'shouldn', 'under', 'hasn', 'ma', 'wasn', 'when', 'themselves', 'same', 'was', 'y', 'out', 'of', "should've", 'what', 'is', 'over', 'isn', "haven't", 'than', 'very', 's', "hadn't", 'whom', 'as', 're', 'am', 'such', 'why', 'against', "you'll", "wouldn't

applying stemming for normalization

In [17]:
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
data['review']=data['review'].apply(simple_stemmer)

splitiing data

In [73]:
#drop the null values in Year
data.dropna(subset=['Year'],inplace=True)
data.dropna(subset = ['sentiment']  , inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4894 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name of Movie  4894 non-null   object 
 1   Year           4894 non-null   float64
 2   Genre          4894 non-null   object 
 3   Rating         4894 non-null   float64
 4   review         4894 non-null   object 
 5   sentiment      4894 non-null   object 
dtypes: float64(2), object(4)
memory usage: 267.6+ KB


In [76]:
norm_train_reviews=data.review[:2447]
norm_train_reviews[0]
norm_test_reviews=data.review[2447:]
norm_test_reviews[2555]

'seri random seemingli insignific theft sister board hous miss lemon quit agit ring light bulb rucksack lighter stethoscop shoe seem rhyme reason miss lemon ask employ great belgian detect hercul poirot look matter poirot see someth far sinist miss lemon could imagin poirot fear confirm one student live board hous found murder poirot bring killer justicebr br hickori dickori dock solid spectacular entri long run poirot seri appreci faith script agatha christi origin stori realiz certain liberti taken appreci effort nonetheless major point mysteri petti theft board hous student rip rucksack cours poirot abil see someth sinist go actual happen except cast student almost pictur damian lewi jessica lloyd standout among group mush alway enjoy david suchet poirot get real kick episod phillip jackson inspector japp paulin moran miss lemon episod real treat miss lemon get screen time usual final enjoy use ever present mous observ activ hostel fun littl play hickori dickori dock titlebr br real

TF-IDF to statistically measure how important a word is in a collection of documents

In [77]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (2447, 493550)
Tfidf_test: (2447, 493550)


LabelBinarizer converts input labels into binary label

In [78]:
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(4894, 1)


In [79]:
train_sentiments=sentiment_data[:2447]
test_sentiments=sentiment_data[2447:]
#make sure that both train and test sentiments are of same shape
print('train_sentiments:',train_sentiments.shape)
print('test_sentiments:',test_sentiments.shape)
#if they are not of same shape then use ravel() function


train_sentiments: (2447, 1)
test_sentiments: (2447, 1)


In [80]:
#fitting the svm for tfidf features
mnb=MultinomialNB()
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()


c:\Users\Ms Links\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [81]:
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

[1 0 1 ... 0 1 1]


In [82]:
#Use accuracy_score function to get the accuracy
print("Tfidf Accuracy Score -> ",accuracy_score(test_sentiments,mnb_tfidf_predict)*100)


Tfidf Accuracy Score ->  68.08336738863915


In [83]:
#Accuracy score for tfidf features
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
#mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)
#if it is throwing error then use ravel() function



mnb_tfidf_score : 0.6808336738863915


In [84]:
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.71      0.62      0.66      1246
    Negative       0.65      0.74      0.70      1201

    accuracy                           0.68      2447
   macro avg       0.68      0.68      0.68      2447
weighted avg       0.68      0.68      0.68      2447

